# Flight Delay Demo - Hybrid ML with Azure Arc

## Setup working directory

The cell below creates our working directory. This will hold our generated scripts.

In [ ]:
import os

project_folder = './scripts'

# Working directory
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

Import and verify the Azure ML SDK version.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Connect to workspace
In the next cell, we create a new Workspace config object using the `<subscription_id>`, `<resource_group_name>`, and `<workspace_name>`. This will fetch the matching Workspace and prompt you for authentication. Please click on the link and input the provided details.

For more information on **Workspace**, please visit: [Microsoft Workspace Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py)

`<subscription_id>` = You can get this ID from the landing page of your Resource Group.

`<resource_group_name>` = This is the name of your Resource Group.

`<workspace_name>` = This is the name of your Workspace.

In [ ]:
from azureml.core.workspace import Workspace

try:    
    # Get instance of the Workspace and write it to config file
    ws = Workspace(
        subscription_id = '<subscription_id>', 
        resource_group = '<resource_group>', 
        workspace_name = '<workspace_name>')

    # Writes workspace config file
    ws.write_config()
    
    print('Library configuration succeeded')
except Exception as e:
    print(e)
    print('Workspace not found')

## Fetch Azure Arc Attached Compute

Fetch the Azure Arc enabled Attached Compute.

For more information on **How to Attach Azure Arc enabled Kubernetes cluster**, please visit: [Microsoft Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-attach-arc-kubernetes)

In [ ]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "<attached_compute_name>"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Please attach the Azure Arc enabled Kubernetes clu...')

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

## Copy files

Copy the necesary files into our project folder.

In [ ]:
import shutil

shutil.copy('flight-delay-train.py', project_folder)
shutil.copy('flightdelayweather_ds_clean.csv', project_folder)

## Create an Experiment

The Experiment constructor allows to create an experiment instance. The constructor takes in the current workspace, which is fetched by calling `Workspace.from_config()` and an experiment name. 

The `experiment.submit()` function is called to send the experiment for execution. The only parameter received by this function is the `AutoMLConfig` object instantiated previously in this module.

For more information on **Experiment**, please visit: [Microsoft Experiment Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py)

In [ ]:
from azureml.core import Experiment

experiment_name = 'flight-delay-exp-arc'
experiment = Experiment(ws, name=experiment_name)

## Write Conda Dependencies file

The dependencies file allows us to define the libraries to be included in the training environment.

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - matplotlib
  - seaborn
  - azureml-defaults

## Create Script Configuration

The `ScriptRunConfig` will setup the training details for the Hybrid ML Training to take place.

For more information on **ScriptRunConfig**, please visit: [Microsoft RunDetails Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py)


In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')
src = ScriptRunConfig(source_directory=project_folder,
                      script='flight-delay-train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

## Show Run Details

The `RunDetails()` function will retrieve the progress, metrics, and tasks for the specified run. They will be displayed by calling the function `show()` over the mentioned object.

For more information on **RunDetails**, please visit: [Microsoft RunDetails Documentation](https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets.rundetails?view=azure-ml-py)


**Note:** Please wait for the execution of the cell to finish before moving forward. (Status should be **Completed**)

In [ ]:
from azureml.widgets import RunDetails

run = experiment.submit(src)
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)